# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [ ]:
slice_name = 'MySliceL2Bridge1'
site = 'TACC'
node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
username = 'centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

## Create and Submit Slice


In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site="MAX")
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    #iface1.set_vlan(vlan='1000')
    
    # Node2
    node2 = slice.add_node(name=node2_name, site="STAR")
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    #iface2.set_vlan(vlan='1000')
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])
    
    #Submit Slice Request
    slice.submit(wait_progress=True)

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

## Get and View the Slice Resources

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

In [ ]:
try:
    node1 = slice.get_node(node1_name)
    iface1 = node1.get_interface(network_name=network_name)
    
    print(f"iface: {iface1.get_name()}")
    
    iface1.set_ip(ip="192.168.1.10", cidr="24")

    print(f"ifname: {iface1.get_os_interface()}")
    print(f"mac: {iface1.get_mac()}")
    stdout, stderr = node1.execute(f'sudo ip addr list {iface1.get_os_interface()}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


### Configure Node2

In [ ]:
try:
    node2 = slice.get_node(node2_name)
    iface1 = node2.get_interface(network_name=network_name)
    
    iface1.set_ip(ip="192.168.1.11", cidr="24")

    print(f"ifname: {iface1.get_os_interface()}")
    print(f"mac: {iface1.get_mac()}")
    stdout, stderr = node2.execute(f'sudo ip addr list {iface1.get_os_interface()}')
    print (f"stdout: {stdout}")
    print (f"stderr: {stderr}")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()